In [1]:
# Call the relevant python packages
import numpy as np
import pandas as pd
import glob, os                          # Operating system module and pathnames

import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.io import ascii, fits
from astropy.table import join, hstack, Table, Column, vstack
from astropy.coordinates import Angle, Latitude, Longitude, SkyCoord

#import SED Fitter module
from sedfitter import fit
from sedfitter import plot
from sedfitter.extinction import Extinction
from sedfitter import plot_params_1d as param1d

# Set colour palette & plot layouts
import seaborn as sns     #; sns.set()
sns.set_context("paper")   # "paper", "notebook", "talk", "poster"

# Output plots in Notebook
#%matplotlib notebook
%matplotlib inline

# Only run the cell below if you need to create the SED format file

In [2]:
# Read in the catalogue
file = '/scratch/s1625982/NGC6822/env/NGC6822_YSOs.csv'
data = ascii.read(file, data_start=1)

# Print the number of YSOs found in the imported table
YSO_num=len(data)
print("The number of YSOs in the table is",YSO_num)

# Set up new columns for the flux
data['J flux']=0.
data['H flux']=0.
data['K flux']=0.
data['36 flux']=0.
data['45 flux']=0.
data['58 flux']=0.
data['80 flux']=0.
data['24 flux']=0.

# Set up new columns for the flux errors
data['J flux err']=0.2
data['H flux err']=0.2
data['K flux err']=0.2
data['36 flux err']=0.2
data['45 flux err']=0.2
data['58 flux err']=0.2
data['80 flux err']=0.2
data['24 flux err']=0.2

# Set up new columns for the integer assigned to each source (0-4).
# NB: this program assumes sources are either valid or upper limits.
# Hence, flags are either 1 (valid) or 3 (upper limit).
# Assume all sources to be valid initially
data['J int']=1
data['H int']=1
data['K int']=1
data['36 int']=1
data['45 int']=1
data['58 int']=1
data['80 int']=1
data['24 int']=1

# Assign the integer describing source reliability
def write_column(idx,name):
    for i in range(len(idx[0])):
        #if source is upper limit write 0 flag
        data[name][idx[0][i]]=0

# Create indices to find which sources are upper limits
idx_J=np.where((data['J data?']==0))
write_column(idx_J,'J int')
idx_H=np.where((data['H data?']==0))
write_column(idx_H,'H int')
idx_K=np.where((data['J data?']==0))
write_column(idx_K,'K int')
idx_36=np.where((data['36 data?']==0))
write_column(idx_36,'36 int')
idx_45=np.where((data['45 data?']==0))
write_column(idx_45,'45 int')
idx_58=np.where((data['58 data?']==0))
write_column(idx_58,'58 int')
idx_80=np.where((data['80 data?']==0))
write_column(idx_80,'80 int')
idx_24=np.where((data['24 data?']==0))
write_column(idx_24,'24 int')

# Hardcode zero points of filters for Vega system.
# Fluxes and their errors are given in mJy
F_0_J, F_0_err_J, m_0_J = 1601000, 0.0, -0.016
F_0_H, F_0_err_H, m_0_H = 1034000, 0.0, 0.018
F_0_K, F_0_err_K, m_0_K = 670300, 0.0, 0.008
F_0_36, F_0_err_36, m_0_36 = 280900, 4100, 0.0
F_0_45, F_0_err_45, m_0_45 = 179700, 2600, 0.0
F_0_58, F_0_err_58,m_0_58 = 115000, 1700, 0.0
F_0_80, F_0_err_80, m_0_80 = 1034000, 900, 0.0
F_0_24, F_0_err_24, m_0_24 = 64900, 110, 0.0

# Define lists for Fv, Fv_0, m_0 etc.
flux_list = ['J flux','H flux','K flux','36 flux','45 flux','58 flux','80 flux','24 flux']
flux_err_list = ['J flux err','H flux err','K flux err','36 flux err','45 flux err','58 flux err','80 flux err','24 flux err']
F_0_list = [F_0_J,F_0_H,F_0_K,F_0_36,F_0_45,F_0_58,F_0_80,F_0_24]
F_0_err_list = [F_0_err_J,F_0_err_H,F_0_err_K,F_0_err_36,F_0_err_45,F_0_err_58,F_0_err_80,F_0_err_24]
m_0_list = [m_0_J,m_0_H,m_0_K,m_0_36,m_0_45,m_0_58,m_0_80,m_0_24]
mag_list = ['Jmag','Hmag','Kmag','k36mag','k45mag','k58mag','k80mag','k24mag']
mag_err_list = ['eJmag','eHmag','eKmag','e36mag','e45mag','e58mag','e80mag','e24mag']

# Write flux and flux error calculations out for each source in each filter
for i in range(len(flux_list)):
        data[flux_list[i]]=F_0_list[i]*np.power(10,0.4*(m_0_list[i]-data[mag_list[i]]))
        data[flux_err_list[i]]=data[flux_list[i]]*np.sqrt((np.square(0.921*data[mag_err_list[i]]))-np.square(F_0_err_list[i]/F_0_list[i]))

# Write out table in the right format for SED fitter
ascii.write(data[['New name','RA','Dec','J int','H int','K int','36 int','45 int','58 int','80 int','24 int',
                  'J flux','J flux err','H flux','H flux err','K flux','K flux err','36 flux','36 flux err',
                  '45 flux','45 flux err','58 flux','58 flux err','80 flux','80 flux err','24 flux','24 flux err']],
            'NGC6822_YSOs_SED_format.txt', format='no_header', fast_writer=False)

# NB: THE USER IS REQUIRED TO FIND AND REPLACE 'INF' IN THE FILE WITH WHATEVER THEY WISH TO USE
#     TO DENOTE UPPER LIMITS (E.G. -9999)

AttributeError: 'builtin_function_or_method' object has no attribute 'read'

# Model Fitting and SED Plotting

The modelling process doesn't seem to like to fit more than ~250 sources at a time without the kernel crashing and won't plot more than ~100 SEDs in one go without claiming that the files you are asking it to create (the actual graphs) don't exist.

Ater a while the whole program won't run at all and you have to save, close and restart.

In [3]:
# Define path to models
model_dir = '/scratch/s1625982/NGC6822/env/environ/models_r06'

# Read in extinction law)
extinction = Extinction.from_file('/scratch/s1625982/NGC6822/env/environ/sedfitter-1.3/examples/kmh94.par',
                                  columns=[0, 3], wav_unit=u.micron, chi_unit=u.cm**2 / u.g)

# Define filters and apertures
filters = ['2J', '2H', '2K', 'I1', 'I2', 'I3', 'I4', 'M1']
apertures = [2., 2., 2., 1.2, 1.2, 1.2, 1.2, 6.] * u.arcsec


# Run the fitting

# Currently (12th June 2019) having to run the fitting and plotting in small batches of 100 - see above.
# Define start and end sources to run - the user must trim the master table to the appropriate limits
start_source = str(501)
end_source = str(584)
SED_input_file = '/scratch/s1625982/NGC6822/env/environ/'+start_source+'to'+end_source+'.txt'
SED_output_file = '/scratch/s1625982/NGC6822/env/environ/output_'+start_source+'to'+end_source+'.fitinfo'
data = ascii.read(SED_input_file, data_start=1)
fit(SED_input_file, filters, apertures, model_dir,
    SED_output_file,
    extinction_law=extinction,
    distance_range=[450., 530.] * u.kpc,    # Distance range here is for NGC 6822
    av_range=[0., 5.])                      # A_V range here is for NGC 6822



from sedfitter import plot
plot(SED_output_file, 'Plots/plots_seds_'+start_source+'to'+end_source)

/scratch/s1625982/NGC6822/env/environ/501to584.txt
/scratch/s1625982/NGC6822/env/environ/output_501to584.fitinfo
 ------------------------------------------------------------
  => Model parameters
 ------------------------------------------------------------

   Models              :  YSO models
   Log[d] stepping     :  0.01
   Number of distances :  8

 ------------------------------------------------------------
  => Reading in convolved fluxes
 ------------------------------------------------------------

   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/2J.fits
   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/2H.fits
   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/2K.fits
   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/I1.fits
   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/I2.fits
   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/I3.fits
   Reading /scrat

# SED Plots Within Chi^2 Range

In [2]:
# plots graphs of the models as above but also draws on models with a chi^2 value greater than that
# of the best fit by the threshold number used in the select_format argument
plot('/scratch/s1625982/NGC6822/env/environ/YSO_SEDs/output_101to200.fitinfo',
     'YSOs_best_chi',
     select_format=('F', 3))

# Parameter Plots

In [13]:
# plots a graph of parameter - defined below
parameter = 'MDISK'

param1d('/scratch/s1625982/NGC6822/env/environ/YSO_SEDs/output_101to200.fitinfo',
        parameter,
        'YSOs_param1d_101to200TEMP')

Delete directory YSOs_param1d_101to200TEMP? [y/[n]] n
Aborting...


SystemExit: 

/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
